In [2]:
import pandas as pd
from os import path

In [3]:
df = pd.read_excel (r'sharedData\200824_Testplan.xlsx', sheet_name='Durchgeführte Messungen')
dfDict = df.to_dict("records")

In [4]:
from mongoTools import connectMongoDB
db = connectMongoDB(True)
db["Testplan"].insert_many(dfDict)

In [7]:
from InFusionTools import convertTime, TIMESTAMPFACTOR
import time
import datetime

dfTestplan = pd.DataFrame(list(db["Testplan"].find()))
for f in dfTestplan["Filename"]:
    f = f.replace("-", "_")
    f = f.replace("c", "k")
    if "vermutlich:" in f:
        f = f.replace("vermutlich: ", "")
    if "Re" in f:
        timestamp = convertTime(f)*TIMESTAMPFACTOR

    else:        
        timestamp = time.mktime(datetime.datetime.strptime(f,
                                                "%d.%m.%Y_%H_%M").timetuple())*TIMESTAMPFACTOR
    dfTestplan["StartID"] = timestamp

ValueError: time data 'vermutlikh: Rekorder_2020_09_03_08_25_35' does not match format 'Rekorder_%Y_%m_%d_%H_%M_%S'